# Phase 3 - Business Intelligence : Visualisation et Dashboard

Ce notebook présente les visualisations finales de l'analyse de sentiment :
- Graphique camembert des sentiments
- WordCloud des tweets négatifs
- Histogrammes temporels
- Analyse des tweets problématiques



In [ ]:
# Importation des bibliothèques
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from src.analyze_tesla_sentiment import TeslaSentimentAnalyzer

print("✅ Bibliothèques importées")



## Étape 1 : Chargement des données analysées


In [ ]:
# Charger les résultats d'analyse
df = pd.read_csv("../data/tesla_sentiment_results.csv")

# Convertir la date en datetime
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])

print(f"📊 {len(df)} tweets analysés chargés")

# Statistiques rapides
print(f"\n📈 Distribution des sentiments :")
print(df['sentiment'].value_counts())



## Étape 2 : Graphique Camembert (Pie Chart) des Sentiments


In [ ]:
# Créer un graphique camembert
sentiment_counts = df['sentiment'].value_counts()

# Couleurs personnalisées
colors = {'positive': '#2E7D32', 'negative': '#C62828', 'neutral': '#757575'}
color_list = [colors.get(sent.lower(), '#757575') for sent in sentiment_counts.index]

fig = go.Figure(data=[go.Pie(
    labels=sentiment_counts.index.str.title(),
    values=sentiment_counts.values,
    hole=0.4,  # Donut chart
    marker_colors=color_list,
    textinfo='label+percent',
    textfont_size=14
)])

fig.update_layout(
    title={
        'text': 'Distribution des Sentiments',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    height=500
)

fig.show()



## Étape 3 : WordCloud des Tweets Négatifs


In [ ]:
# Filtrer les tweets négatifs
negative_tweets = df[df['sentiment'] == 'negative']['text_cleaned'].dropna()

if len(negative_tweets) > 0:
    # Combiner tous les tweets négatifs
    text = ' '.join(negative_tweets.astype(str))
    
    # Créer le WordCloud
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        colormap='Reds',
        max_words=100,
        relative_scaling=0.5,
        collocations=False
    ).generate(text)
    
    # Afficher
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Mots les plus fréquents dans les tweets négatifs', 
              fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
else:
    print("Aucun tweet négatif disponible")



## Étape 4 : Histogramme Temporel


In [ ]:
# Histogramme par date
if 'date' in df.columns:
    df['date_only'] = df['date'].dt.date
    tweets_by_date = df.groupby('date_only').size().reset_index(name='count')
    
    fig = px.bar(
        tweets_by_date,
        x='date_only',
        y='count',
        labels={'date_only': 'Date', 'count': 'Nombre de tweets'},
        color='count',
        color_continuous_scale='Reds',
        title='Volume de tweets par jour'
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Nombre de tweets",
        height=400,
        showlegend=False
    )
    
    fig.show()



In [ ]:
# Histogramme par heure de la journée
if 'date' in df.columns:
    df['hour'] = df['date'].dt.hour
    tweets_by_hour = df.groupby('hour').size().reset_index(name='count')
    
    fig = px.bar(
        tweets_by_hour,
        x='hour',
        y='count',
        labels={'hour': 'Heure de la journée', 'count': 'Nombre de tweets'},
        color='count',
        color_continuous_scale='Blues',
        title='Volume de tweets par heure de la journée'
    )
    
    fig.update_layout(
        xaxis_title="Heure",
        yaxis_title="Nombre de tweets",
        height=400,
        showlegend=False
    )
    
    fig.show()



## Étape 5 : Analyse des Tweets les Plus Négatifs


In [ ]:
# Obtenir les 5 tweets les plus négatifs
analyzer = TeslaSentimentAnalyzer()
top_negative = analyzer.get_top_negative_tweets(df, n=5)

print("🔴 Top 5 tweets les plus négatifs :\n")
for idx, (_, row) in enumerate(top_negative.iterrows(), 1):
    print(f"{'='*80}")
    print(f"Tweet #{idx}")
    print(f"Polarité: {row['polarity']:.3f} | Score négatif VADER: {row['vader_neg']:.3f}")
    print(f"Likes: {row.get('likes', 0)} | RT: {row.get('retweets', 0)}")
    print(f"\nTexte original:\n{row['text']}")
    
    # Détecter le sarcasme
    sarcasm_indicators = analyzer.detect_sarcasm_indicators(row['text'])
    if sarcasm_indicators:
        print(f"\n⚠️  Indicateurs de sarcasme détectés: {', '.join(sarcasm_indicators)}")
    print()



## Étape 6 : Visualisations Complémentaires


In [ ]:
# Distribution des polarités par sentiment
fig, ax = plt.subplots(figsize=(12, 6))

for sentiment in df['sentiment'].unique():
    subset = df[df['sentiment'] == sentiment]
    ax.hist(subset['polarity'], bins=30, alpha=0.6, label=sentiment.title(), 
           color=colors.get(sentiment.lower(), 'gray'))

ax.axvline(x=0.1, color='green', linestyle='--', alpha=0.5, label='Seuil positif')
ax.axvline(x=-0.1, color='red', linestyle='--', alpha=0.5, label='Seuil négatif')
ax.set_xlabel('Polarité', fontsize=12)
ax.set_ylabel('Fréquence', fontsize=12)
ax.set_title('Distribution des Polarités par Sentiment', fontsize=16, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()



In [ ]:
# Analyse des interactions (Likes, RT) par sentiment
if 'likes' in df.columns and 'retweets' in df.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Boxplot des likes par sentiment
    df.boxplot(column='likes', by='sentiment', ax=ax1)
    ax1.set_title('Distribution des Likes par Sentiment', fontweight='bold')
    ax1.set_xlabel('Sentiment')
    ax1.set_ylabel('Nombre de Likes')
    
    # Boxplot des retweets par sentiment
    df.boxplot(column='retweets', by='sentiment', ax=ax2)
    ax2.set_title('Distribution des Retweets par Sentiment', fontweight='bold')
    ax2.set_xlabel('Sentiment')
    ax2.set_ylabel('Nombre de Retweets')
    
    plt.suptitle('')
    plt.tight_layout()
    plt.show()



## Conclusion

✅ **Visualisations créées avec succès !**

Toutes les visualisations ont été générées :
- ✅ Graphique camembert des sentiments
- ✅ WordCloud des tweets négatifs
- ✅ Histogrammes temporels (par jour et par heure)
- ✅ Analyse des tweets les plus négatifs
- ✅ Visualisations complémentaires

**Pour un dashboard interactif**, exécutez :
```bash
streamlit run src/tesla_dashboard.py
```

